In [38]:
import os
import pandas as pd
import numpy as np
import csv
import re

from sqlalchemy import inspect
from sqlalchemy import create_engine, MetaData, Table, text, select
from sqlalchemy.orm import sessionmaker

import logging
logging.basicConfig(level=logging.DEBUG)

### Accessing the database

In [2]:
# create engine
engine = create_engine("postgresql://adityakanthale@localhost/imdb-yin")
args, kwargs = engine.dialect.create_connect_args(engine.url)

# load metadata
metadata = MetaData()
metadata.create_all(engine)

# inspector to view columns and database
inspector = inspect(engine)

In [3]:
# load table
cast_info_tb = Table('cast_info', metadata)

#create session
Session = sessionmaker(bind=engine)
session = Session()

### Pre-processing (Generic)

In [4]:
def write_table_to_csv(table_name, out_dir='./out'):

    with engine.connect() as conn:
        query = text(f"SELECT * FROM {table_name}")
        rs = conn.execute(query)
        with open(f"{out_dir}/{table_name}.csv", 'w') as f:
            outcsv = csv.writer(f)
            outcsv.writerow(rs.keys())
            outcsv.writerows(rs)

In [115]:
write_table_to_csv('ROLE_TYPE')
write_table_to_csv('MOVIE_COMPANIES')
write_table_to_csv('CAST_INFO')

In [5]:
def write_table_to_df(table_name):
    df = pd.DataFrame()
    with engine.connect() as conn:
        df = pd.read_sql_table(table_name=str(table_name).lower(), con=conn)
        df.head()
    return df

In [6]:
role_type_df = write_table_to_df('ROLE_TYPE')
movie_companies_df = write_table_to_df('MOVIE_COMPANIES')
cast_info_df = write_table_to_df('CAST_INFO')

In [122]:
cast_info_fl = pd.read_csv("./out/CAST_INFO.csv")

In [13]:
note_s = '(2013) (USA) (all media)'.split(') ')
note_s = [s.strip('(').strip(')') for s in note_s]
note_s

['2013', 'USA', 'all media']

In [48]:
t = [val for val in split_note_df.apply(lambda x: x[-1] if x else None).unique()]

In [72]:
def convert_to_valid(arr):

  year_pattern = r"^\d{4}(-\d{4})?$"
  area_len = 40

  res = [None, None, None]
  if len(arr) > 0 and re.match(year_pattern, arr[0]): res[0] = arr[0]
  if len(arr) > 1 and len(arr[1]) <= area_len: res[1] = arr[1]
  if len(arr) > 2: res[2] = arr[2]
  return res

def split_note(note):
    if note:
        note_s = note.split(') ')
        note_vals = [s.strip('(').strip(')') for s in note_s]
        note_vals = convert_to_valid(note_vals)
        return note_vals
    return [None, None, None]

split_note_df = movie_companies_df['note'].apply(split_note)

columns=['year', 'area', 'note']
movie_companies_df[columns] = pd.DataFrame(split_note_df.to_list(), columns=columns)
movie_companies_df.head()

,id,movie_id,company_id,company_type_id,note,year,area
0,1,2,1,1,TV,2006,USA
1,2,2,1,1,TV,2006,worldwide
2,3,11,2,1,all media,2012,worldwide
3,4,44,3,1,all media,2013,USA
4,5,50,4,1,TV,2011,UK


In [117]:
min_year = 1850
max_year = 2023

def get_bin_map(yr):
    return (yr-min_year)//10

def get_bin_yr(yr):
    if yr:
        if '-' in yr:
            yr = yr[:yr.index('-')]
        yr = int(yr)
        return get_bin_map(yr)

In [118]:
mc_copy = movie_companies_df.copy()
mc_copy['year'] = mc_copy['year'].apply(func=get_bin_yr)
print(len(mc_copy))
print(len(mc_copy[mc_copy['year'].between(0,(max_year-min_year)//10)]))

2609129
1109958


In [122]:
mc_copy[mc_copy['year'].between(0,(max_year-min_year)//10)].to_csv('./out/movie_copmpanies_processed.csv')

0          15.0
1          15.0
2          16.0
3          16.0
4          16.0
           ... 
2561798     9.0
2563907     9.0
2578898    15.0
2599679    10.0
2608909    10.0
Name: year, Length: 1109958, dtype: float64

In [79]:
min_year = 1000000000
max_year = -1000000000
for yr in movie_companies_df.year.unique():
    if yr and '-' not in yr:
        yr = int(yr)
        min_year = min(min_year, yr)
        max_year = max(max_year, yr)
print(min_year, max_year)



1003 9100


Data Pre-processing:

1. Read join tables
2. Locate foreign key, sensitive attributes
1. identify continuous values
    1. bin them
    2. add these as new columns

Routine:

1. Find distribution of attributes highly-correlated with the sensitive attrs
2. Find join over samples with threshold a threshold for minority class???
3. Maximal uncovered Pattern

### Alternate methods

# to read table